In [65]:
import requests 
from bs4 import BeautifulSoup 
import json
import re
import concurrent.futures
page_list=[page for page in range(1,4314)]

In [66]:
def crawl_url(page):
    list_urls=[]
    
    data= { "searchMode":"",
            "searchType":"text",
            "ExtFilter":"",
            "sorttype":"1",
            "keyword":"政治",
            "rangedate":"[20140101 TO 20171215999:99]",
            "totalpage":"43137",
            "page":"%s"%page }

    resp = requests.post("https://tw.appledaily.com/search",data=data)
    soup = BeautifulSoup(resp.text,"html5lib")
    select = soup.select("div.tbb > h2 > a")
    for s in select:
        url = s.get("href")
        list_urls.append(url)
    return list_urls

list_a=[]
with concurrent.futures.ThreadPoolExecutor(max_workers=80) as executor:
    for data in executor.map(crawl_url, page_list):
        list_a.extend(data)     
with open("appledaily_urls.txt","w") as f:
    f.write(json.dumps(list_a))

In [ ]:
with open("appledaily_urls.txt","r") as f:
    r = f.read()
urls = r.replace("]","").replace("[","").replace('"','').split(",")

list_a_new = [] 
for i in urls:
    i = i.strip()
    if i.startswith("https://tw.news.appledaily.com/politics/realtime"):
        list_a_new.append(i.strip())


In [92]:
def crawl_article(url):
    list_article=[]
    resp_co = requests.get(url)
    soup = BeautifulSoup(resp_co.text,"html5lib")
    article = {}
    try:
        ### datetime
        datetime = soup.select("div.ndArticle_creat")[0].text
        article["datetime"] = re.findall("出版時間：([2][0][1][4567]/[01][\d]/[012][\d])",datetime)[0].replace("/","-")    
        ### title
        title = soup.select("h1")
        article["title"] = title[0].text.replace("\u3000","").replace("\u200b","")
        ### media
        article["media"]="appledaily"
        ### category
        article["category"]="politics"
        ### url
        article["url"]=url
        ### reporter
        article["reporter"] = None
        ### location
        article["location"] = None
        ### content
        article["content"] = soup.select("div.ndArticle_margin > p")[0].text
        ### hash
        article["hash"] = hash(soup.select("div.ndArticle_margin > p")[0].text)
        ### id
        article["id"] = str(url.split("/")[-1])
    except:
        pass
        
    list_article.append(article)
    return list_article

In [ ]:
list_b=[]
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    for data in executor.map(crawl_article, list_a_new):
        print("finish")
        list_b.extend(data) 

In [96]:
len(list_b)

12257